In [13]:
import requests

import pandas as pd
import sqlite3

from tqdm import tqdm

In [14]:
reqRegiones = requests.get("https://api-colombia.com/api/v1/Region").json()

assert dfRegiones.isnull().sum().sum() == 0, "❌ Null values detected in Regions"
assert dfRegiones.duplicated().sum() == 0, "❌ Duplicate rows detected in Regions"
assert dfRegiones["id"].nunique() == 6, "❌ There must be exactly 6 regions"

In [15]:
regiones = [{k: v for k, v in ele.items() if k != "departments"} for ele in reqRegiones]
dfRegiones = pd.DataFrame(regiones)

In [16]:
reqDepartaments = [requests.get(f"https://api-colombia.com/api/v1/Region/{region}/departments").json() for region in tqdm(range(1, 7))]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.34s/it]


In [17]:
dfsDeparments = []

for region in tqdm(reqDepartaments):
    deparment = [{k: v for k, v in ele.items() if v!= None and not isinstance(v, dict)} for ele in region]
    dfsDeparments.append(pd.DataFrame(deparment))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 808.75it/s]


In [18]:
dfDeparments = pd.concat(dfsDeparments)
total_Region_Dep = dfDeparments['regionId'].nunique()
if total_Region_Dep != 6:
    raise Exception("The total of regions are 6")

In [ ]:
# 1. Nulls
if dfDeparments.isnull().sum().sum() > 0:
    raise Exception("❌ Null values found in Departments")

# 2. Duplicates
if dfDeparments.duplicated().sum() > 0:
    raise Exception("❌ Duplicate rows found in Departments")


In [20]:
conn = sqlite3.connect('applications.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS Departmnets')
cursor.execute('DROP TABLE IF EXISTS Regions')
dfDeparments.to_sql('Departmnets', conn, if_exists='replace', index=False)
dfRegiones.to_sql('Regions', conn, if_exists='replace', index=False)

6